In [ ]:
import pandas as pd

In [ ]:
import requests
import time
import json

API_KEY = "6993505f971534.72691327"
BASE_URL = "https://eodhd.com/api/news"

years = range(2017, 2026)
all_articles = []

for year in years:
    from_date = f"{year}-01-01"
    to_date = f"{year}-12-31"
    offset = 0
    
    while True:
        params = {
            "api_token": API_KEY,
            "fmt": "json",
            "t": "DAX",
            "from": from_date,
            "to": to_date,
            "limit": 100,
            "offset": offset
        }
        
        response = requests.get(BASE_URL, params=params)
        data = response.json()
        
        if not data:
            break
        
        all_articles.extend(data)
        offset += 100
        
        print(f"{year} - Pulled {offset} articles...")
        time.sleep(0.5)  # throttle safety

print(f"Total articles collected: {len(all_articles)}")

with open("SP500_news_2017_2025.json", "w") as f:
    json.dump(all_articles, f)

print("Download complete.")

In [ ]:
import requests
import time
import json

API_KEY = "6993505f971534.72691327"
BASE_URL = "https://eodhd.com/api/news"

years = range(2017, 2026)
all_articles = []

for year in years:
    from_date = f"{year}-01-01"
    to_date = f"{year}-12-31"
    offset = 0
    
    while True:
        params = {
            "api_token": API_KEY,
            "fmt": "json",
            "t": "DAX",
            "from": from_date,
            "to": to_date,
            "limit": 100,
            "offset": offset
        }
        
        response = requests.get(BASE_URL, params=params)
        data = response.json()
        
        if not data:
            break
        
        all_articles.extend(data)
        offset += 100
        
        print(f"{year} - Pulled {offset} articles...")
        time.sleep(0.5)  # throttle safety

print(f"Total articles collected: {len(all_articles)}")

with open("SP500_news_2017_2025.json", "w") as f:
    json.dump(all_articles, f)

print("Download complete.")

In [15]:
import requests
import time
import json
import csv

# -------------------------
# CONFIG
# -------------------------
API_KEY = "6993505f971534.72691327"

# Largest S&P 500 companies by market cap

tickers = [
    "SAP",
    "SIE",
    "ALV",
    "AIR",
    "BAS",
    "BAYN",
    "MBG",
    "BMW",
    "DTE",
    "DBK"
]

years = range(2017, 2026)  # 2017 through 2025
BASE_URL = "https://eodhd.com/api/news"
all_articles = []
article_ids = set()  # for deduplication
MAX_ARTICLES_PER_YEAR = 500  # cap per ticker per year

# -------------------------
# FUNCTION TO PULL NEWS
# -------------------------
def pull_news_for_ticker_year(ticker, year):
    from_date = f"{year}-01-01"
    to_date = f"{year}-12-31"
    offset = 0
    year_articles = []

    while len(year_articles) < MAX_ARTICLES_PER_YEAR:
        params = {
            "api_token": API_KEY,
            "fmt": "json",
            "s": ticker,
            "from": from_date,
            "to": to_date,
            "limit": 100,
            "offset": offset
        }
        response = requests.get(BASE_URL, params=params)

        try:
            data = response.json()
        except requests.JSONDecodeError:
            print(f"Warning: JSON decode error for {ticker} {year} offset {offset}. Retrying...")
            time.sleep(2)
            continue  # Retry this request

        if not data:
            break

        for article in data:
            unique_id = article.get("link") or article.get("title")
            if unique_id and unique_id not in article_ids:
                article_ids.add(unique_id)
                year_articles.append(article)
                if len(year_articles) >= MAX_ARTICLES_PER_YEAR:
                    break

        offset += len(data)
        print(f"{ticker} {year} - collected {len(year_articles)} articles so far...")
        time.sleep(0.5)

    return year_articles


# -------------------------
# MAIN LOOP
# -------------------------
for ticker in tickers:
    print(f"Starting ticker: {ticker}")
    for year in years:
        articles = pull_news_for_ticker_year(ticker, year)
        all_articles.extend(articles)
    print(f"Finished ticker: {ticker}, total articles collected: {len(all_articles)}\n")

print(f"TOTAL articles collected for all tickers: {len(all_articles)}")

# -------------------------
# SAVE JSON
# -------------------------
json_filename = "Stocks_DAX_biggest_news_2017_2025_capped.json"
with open(json_filename, "w", encoding="utf-8") as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=2)
print(f"JSON saved as {json_filename}")

# -------------------------
# SAVE CSV
# -------------------------
csv_filename = "Stocks_DAX_biggest_news_2017_2025_capped.csv"

if all_articles:
    keys = all_articles[0].keys()
    with open(csv_filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        for article in all_articles:
            writer.writerow(article)
    print(f"CSV saved as {csv_filename}")

Starting ticker: SAP
SAP 2017 - collected 10 articles so far...
SAP 2018 - collected 15 articles so far...
SAP 2019 - collected 7 articles so far...
SAP 2020 - collected 19 articles so far...
SAP 2021 - collected 100 articles so far...
SAP 2021 - collected 200 articles so far...
SAP 2021 - collected 264 articles so far...
SAP 2022 - collected 100 articles so far...
SAP 2022 - collected 200 articles so far...
SAP 2022 - collected 257 articles so far...
SAP 2023 - collected 100 articles so far...
SAP 2023 - collected 200 articles so far...
SAP 2023 - collected 300 articles so far...
SAP 2023 - collected 320 articles so far...
SAP 2024 - collected 100 articles so far...
SAP 2024 - collected 200 articles so far...
SAP 2024 - collected 276 articles so far...
SAP 2025 - collected 100 articles so far...
SAP 2025 - collected 200 articles so far...
SAP 2025 - collected 300 articles so far...
SAP 2025 - collected 400 articles so far...
SAP 2025 - collected 500 articles so far...
Finished ticker:

In [14]:
import requests
API_KEY = "6993505f971534.72691327"
params = {
    "api_token": API_KEY,
    "fmt": "json",
    "s": "SAP",
    "from": "2023-01-01",
    "to": "2023-12-31",
    "limit": 100
}
response = requests.get("https://eodhd.com/api/news", params=params)
print(response.status_code)
print(response.text[:500])  # see first 500 chars



200
[{"date":"2023-12-29T15:11:42+00:00","title":"The Next Big Thing: 3 Tech Stocks Ready for a 500% Leap by 2027","content":"In an era where technological advancements wield unprecedented influence, the hunt for the following game-changing tech stocks intensifies. Imagine a landscape where three tech juggernauts illuminate the path to a staggering 5X surge by 2027. Operating at the forefront of semiconductor materials, systems, and application software, these companies aren\u2019t just names on a s


In [ ]:
import requests
import time
import json
import csv

# -------------------------
# CONFIG
# -------------------------
API_KEY = "6993505f971534.72691327"

# Largest S&P 500 companies by market cap
tickers = [
    "AAPL.US", "MSFT.US", "AMZN.US", "GOOGL.US", "META.US",
    "TSLA.US", "NVDA.US", "JPM.US", "JNJ.US"
]

years = range(2017, 2026)  # 2017 through 2025
BASE_URL = "https://eodhd.com/api/news"
all_articles = []
article_ids = set()  # for deduplication
MAX_ARTICLES_PER_YEAR = 300  # cap per ticker per year

# -------------------------
# FUNCTION TO PULL NEWS
# -------------------------
def pull_news_for_ticker_year(ticker, year):
    from_date = f"{year}-01-01"
    to_date = f"{year}-12-31"
    offset = 0
    year_articles = []

    while len(year_articles) < MAX_ARTICLES_PER_YEAR:
        params = {
            "api_token": API_KEY,
            "fmt": "json",
            "s": ticker,
            "from": from_date,
            "to": to_date,
            "limit": 100,
            "offset": offset
        }
        response = requests.get(BASE_URL, params=params)

        try:
            data = response.json()
        except requests.JSONDecodeError:
            print(f"Warning: JSON decode error for {ticker} {year} offset {offset}. Retrying...")
            time.sleep(2)
            continue  # Retry this request

        if not data:
            break

        for article in data:
            unique_id = article.get("link") or article.get("title")
            if unique_id and unique_id not in article_ids:
                article_ids.add(unique_id)
                year_articles.append(article)
                if len(year_articles) >= MAX_ARTICLES_PER_YEAR:
                    break

        offset += len(data)
        print(f"{ticker} {year} - collected {len(year_articles)} articles so far...")
        time.sleep(0.5)

    return year_articles


# -------------------------
# MAIN LOOP
# -------------------------
for ticker in tickers:
    print(f"Starting ticker: {ticker}")
    for year in years:
        articles = pull_news_for_ticker_year(ticker, year)
        all_articles.extend(articles)
    print(f"Finished ticker: {ticker}, total articles collected: {len(all_articles)}\n")

print(f"TOTAL articles collected for all tickers: {len(all_articles)}")

# -------------------------
# SAVE JSON
# -------------------------
json_filename = "Stocks_biggest_news_2017_2025_capped.json"
with open(json_filename, "w", encoding="utf-8") as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=2)
print(f"JSON saved as {json_filename}")

# -------------------------
# SAVE CSV
# -------------------------
csv_filename = "Stocks_biggest_news_2017_2025_capped.csv"

if all_articles:
    keys = all_articles[0].keys()
    with open(csv_filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        for article in all_articles:
            writer.writerow(article)
    print(f"CSV saved as {csv_filename}")


In [ ]:
SP500_news = pd.read_csv("../data/SP500_news_2017_2025")